## Претурање вода

Имате на располагање 3 садови. Нивниот капацитет е 5, 8 и 10 соодветно.  На почетокот сите садови се празни. Задачата е да претурите 1 литар во садот од 5 литри. Можете да претурате вода меѓу садовите, но не знаете точно колку претурате бидејќи садовите не се обележани. Единствено може да го испразните садот од кој претурате или да го наполните до врв садот во кој претурате. Постои и чешма од која може да надополните некој сад до врвот. Секогаш може да ја истурите водата од некој сад. Скратено: имате на располагање 3 садови. Нивниот капацитет е (x, y, z) соодветно. Почетната состојба е (x_0, y_0, z_0). Задачата е да стигнете до посакуваната состојба.

Дополнително, секој нов наточен литар се казнува со 1 поен (еквивалентно на 1 потег), а секој истурен литар се казнува со 3 поени (еквивалетно на 3 потези).

In [37]:
from collections import deque
from itertools import combinations

In [43]:
def transfer_to_other_glass(source, sink, sink_capacity):
    return max(source - (sink_capacity - sink), 0), min(sink + source, sink_capacity), 0

In [45]:
def expand_state(state):
    capacities = (5, 8, 10)
    glass_0, glass_1, glass_2 = state

    new_states = []

    if glass_0 < capacities[0]:
        weight = capacities[0] - glass_0
        new_states.append(((capacities[0], glass_1, glass_2), weight))
    if glass_1 < capacities[1]:
        weight = capacities[1] - glass_1
        new_states.append(((glass_0, capacities[1], glass_2), weight))
    if glass_2 < capacities[2]:
        weight = capacities[2] - glass_2
        new_states.append(((glass_0, glass_1, capacities[2]), weight))
    
    if glass_0 > 0:
        weight = glass_0
        new_states.append(((0, glass_1, glass_2), weight))
    if glass_1 > 0:
        weight = glass_1
        new_states.append(((glass_0, 0, glass_2), weight))
    if glass_2 > 0:
        weight = glass_2
        new_states.append(((glass_0, glass_1, 0), glass_2))

    new_source, new_sink, weight = transfer_to_other_glass(glass_0, glass_1, capacities[1])
    new_states.append(((new_source, new_sink, glass_2), weight))

    new_source, new_sink, weight = transfer_to_other_glass(glass_1, glass_0, capacities[0])
    new_states.append(((new_sink, new_source, glass_2), weight))

    new_source, new_sink, weight = transfer_to_other_glass(glass_0, glass_2, capacities[2])
    new_states.append(((new_source, glass_1, new_sink), weight))

    new_source, new_sink, weight = transfer_to_other_glass(glass_2, glass_0, capacities[0])
    new_states.append(((new_sink, glass_1, new_source), weight))

    new_source, new_sink, weight = transfer_to_other_glass(glass_1, glass_2, capacities[2])
    new_states.append(((glass_0, new_source, new_sink), weight))

    new_source, new_sink, weight = transfer_to_other_glass(glass_2, glass_1, capacities[1])
    new_states.append(((glass_0, new_sink, new_source), weight))
    
    return new_states
expand_state((0, 5, 10))

[((5, 5, 10), 5),
 ((0, 8, 10), 3),
 ((0, 0, 10), 5),
 ((0, 5, 0), 10),
 ((0, 5, 10), 0),
 ((5, 0, 10), 0),
 ((0, 5, 10), 0),
 ((5, 5, 5), 0),
 ((0, 5, 10), 0),
 ((0, 8, 7), 0)]

In [40]:
def search_path(initial_state, goal_state):
    visited = {initial_state}
    states_queue = deque([[initial_state]])
    while states_queue:
        states_list = states_queue.popleft()
        state_to_expand = states_list[-1]
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state == goal_state:
                    return states_list + [next_state]
                visited.add(next_state)
                states_queue.append(states_list + [next_state])
    return []

In [27]:
search_path((0, 0, 0), (1, 0, 0))

[(0, 0, 0),
 (0, 8, 0),
 (5, 3, 0),
 (0, 3, 0),
 (3, 0, 0),
 (3, 8, 0),
 (3, 0, 8),
 (1, 0, 10),
 (1, 0, 0)]

In [23]:
def visualise_path(path):
    for states in zip(path, path[1:]):
        old_state, new_state = states
        print(old_state)
        print(tuple(map(lambda x, y: x - y, new_state, old_state)), 'change')
        print(new_state)
        print()

In [28]:
initial_state = (0, 0, 0)
goal_state = (1, 0, 0)
path = search_path(initial_state, goal_state)
print(path)
visualise_path(path)

[(0, 0, 0), (0, 8, 0), (5, 3, 0), (0, 3, 0), (3, 0, 0), (3, 8, 0), (3, 0, 8), (1, 0, 10), (1, 0, 0)]
(0, 0, 0)
(0, 8, 0) change
(0, 8, 0)

(0, 8, 0)
(5, -5, 0) change
(5, 3, 0)

(5, 3, 0)
(-5, 0, 0) change
(0, 3, 0)

(0, 3, 0)
(3, -3, 0) change
(3, 0, 0)

(3, 0, 0)
(0, 8, 0) change
(3, 8, 0)

(3, 8, 0)
(0, -8, 8) change
(3, 0, 8)

(3, 0, 8)
(-2, 0, 2) change
(1, 0, 10)

(1, 0, 10)
(0, 0, -10) change
(1, 0, 0)



In [50]:
import heapq
def uniform_cost_search(starting_vertex, goal_vertex):
    number_of_expanded_nodes = 0
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue)
        vertex_to_expand = vertex_list[-1]
        number_of_expanded_nodes = number_of_expanded_nodes + 1
        if vertex_to_expand == goal_vertex:
            return weight, vertex_list, number_of_expanded_nodes
        if vertex_to_expand in expanded:
            continue
        for neighbour, new_weight in expand_state(vertex_to_expand):
            if neighbour not in expanded:
                heapq.heappush(queue, (weight + new_weight, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)

In [51]:
uniform_cost_search(initial_state, goal_state)

(21,
 [(0, 0, 0),
  (0, 0, 10),
  (0, 8, 2),
  (2, 8, 0),
  (2, 0, 8),
  (0, 2, 8),
  (5, 2, 3),
  (0, 7, 3),
  (0, 8, 3),
  (0, 1, 10),
  (0, 1, 0),
  (0, 0, 1),
  (1, 0, 0)],
 1612)

## Пивара

* се избират шишињата со код 2
* 0 = празно место
* 3х2 област на земање шишиња или 2х3
* роботот да ги издвои шишињата со код 2 во најмалку чекори
* 1 чекор е да се претвори матрицата од 2 во 0

In [119]:
g = (
    (0, 3, 2, 0, 2),
    (2, 1, 2, 2, 2),
    (2, 2, 0, 2, 2),
    (0, 2, 2, 1, 2),
    (1, 2, 1, 3, 0)
)
list(list(row[:3]) for row in g[0:2])
list(list(row[:2]) for row in g[0:3])
list(list(row) for row in g)


[[0, 3, 2, 0, 2],
 [2, 1, 2, 2, 2],
 [2, 2, 0, 2, 2],
 [0, 2, 2, 1, 2],
 [1, 2, 1, 3, 0]]

initial_state = g

In [120]:
def find_number_of_twos(state):
    number = 0
    for row in state:
        for value in row:
            if value == 2:
                number += 1
    return number

find_number_of_twos(g) // 6

2

In [121]:
def h(state):
    return find_number_of_twos(state) // 6

In [122]:
def check_end(state):
    return all(value != 2 for row in state for value in row)

check_end(g)

False

In [123]:
def apply_move(state, r0, c0, h, w):
    changed = False
    grid = list(list(row) for row in state)
    for i in range(r0, r0 + h):
        for j in range(c0, c0 + w):
            if grid[i][j] == 2:
                changed = True
                grid[i][j] = 0
    if changed:
        return tuple(tuple(row) for row in grid)
    else:
        None

apply_move(g, 0, 0, 2, 3)

((0, 3, 0, 0, 2),
 (0, 1, 0, 2, 2),
 (2, 2, 0, 2, 2),
 (0, 2, 2, 1, 2),
 (1, 2, 1, 3, 0))

In [124]:
def expand_state(state):
    new_states = []
    R, C = len(state), len(state[0])

    for r0 in range(R - 3 + 1):
        for c0 in range(C - 2 + 1):
            ns = apply_move(state, r0, c0, 3, 2)
            if ns is not None:
                new_states.append(ns)

    for r0 in range(R - 2 + 1):
        for c0 in range(C - 3 + 1):
            ns = apply_move(state, r0, c0, 2, 3)
            if ns is not None:
                new_states.append(ns)

    return new_states


In [125]:
def depth_first_search(initial_state):
    
    queue = deque([initial_state])
    visited = {initial_state}
    
    while queue:
        state_to_expand = queue.popleft()
        
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if check_end(next_state):
                    return next_state
                queue.appendleft(next_state)
                visited.add(next_state)
depth_first_search(g)

((0, 3, 0, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 0, 0, 0, 0),
 (0, 0, 0, 1, 0),
 (1, 0, 1, 3, 0))

In [126]:
def a_star_search(starting_vertex, heuristic_function, alpha=1):
    number_of_expanded_nodes = 0
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight_tuple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tuple
        vertex_to_expand = vertex_list[-1]
        number_of_expanded_nodes = number_of_expanded_nodes + 1
        if check_end(vertex_to_expand):
            return current_path_weight, vertex_list, number_of_expanded_nodes + 1
        if vertex_to_expand in expanded:
            continue
        for neighbour in expand_state(vertex_to_expand):
            new_weight = 1
            if neighbour not in expanded:
                heuristic = heuristic_function(neighbour)
                path_weight = current_path_weight + new_weight
                a_star_weight = path_weight + alpha * heuristic
                heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)


In [127]:
a_star_search(g, h)

(4,
 [((0, 3, 2, 0, 2),
   (2, 1, 2, 2, 2),
   (2, 2, 0, 2, 2),
   (0, 2, 2, 1, 2),
   (1, 2, 1, 3, 0)),
  ((0, 3, 0, 0, 0),
   (2, 1, 0, 0, 0),
   (2, 2, 0, 2, 2),
   (0, 2, 2, 1, 2),
   (1, 2, 1, 3, 0)),
  ((0, 3, 0, 0, 0),
   (0, 1, 0, 0, 0),
   (0, 0, 0, 2, 2),
   (0, 0, 2, 1, 2),
   (1, 2, 1, 3, 0)),
  ((0, 3, 0, 0, 0),
   (0, 1, 0, 0, 0),
   (0, 0, 0, 0, 0),
   (0, 0, 0, 1, 0),
   (1, 2, 1, 3, 0)),
  ((0, 3, 0, 0, 0),
   (0, 1, 0, 0, 0),
   (0, 0, 0, 0, 0),
   (0, 0, 0, 1, 0),
   (1, 0, 1, 3, 0))],
 2978)

In [ ]:
def grid_to_mask(grid):
    R, C = len(grid), len(grid[0])
    mask = 0
    for r in range(R):
        for c in range(C):
            if grid[r][c] == 2:
                bit = r * C + c
                mask |= (1 << bit)
    return mask


In [ ]:
def rectangle_masks(R, C):
    moves = []  # (rect_mask, (r0,c0,h,w))
    for h, w in [(3, 2), (2, 3)]:
        for r0 in range(R - h + 1):
            for c0 in range(C - w + 1):
                m = 0
                for r in range(r0, r0 + h):
                    for c in range(c0, c0 + w):
                        bit = r * C + c
                        m |= (1 << bit)
                moves.append((m, (r0, c0, h, w)))
    return moves


In [ ]:
def apply_move_mask(state_mask, rect_mask):
    new_mask = state_mask & ~rect_mask
    return new_mask if new_mask != state_mask else None  # None if no 2 removed


In [ ]:
from collections import deque

def bfs_min_moves(grid):
    R, C = len(grid), len(grid[0])
    start = grid_to_mask(grid)
    if start == 0:
        return []

    moves = rectangle_masks(R, C)

    q = deque([start])
    parent = {start: None}
    parent_move = {start: None}

    while q:
        s = q.popleft()
        for rect_mask, mv in moves:
            ns = apply_move_mask(s, rect_mask)
            if ns is None:
                continue
            if ns in parent:
                continue
            parent[ns] = s
            parent_move[ns] = mv

            if ns == 0:
                # reconstruct
                path = []
                cur = ns
                while parent_move[cur] is not None:
                    path.append(parent_move[cur])
                    cur = parent[cur]
                path.reverse()
                return path

            q.append(ns)

    return None


In [ ]:
bfs_min_moves(g)